In [1]:
import os
os.chdir('/home/jupyter-nafisha/normal-abnormal-multitask/main')

In [2]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
checkpoint_path= '/home/jupyter-nafisha/normal-abnormal-multitask/main/last_model.pth'

In [15]:
import os
import cv2
import torch
from PIL import Image
import pandas as pd
import numpy as np
# from utils import encode_disease
from torch.utils.data import Dataset
from config import disease2id
from utils import has_valid_bbox

class XrayDataset(Dataset):
    def __init__(self, img_dir, csv_path, transform_bbox = None, transform_nobbox = None):
        self.df = pd.read_csv(csv_path)
        self.img_dir = img_dir
        self.transform_bbox = transform_bbox
        self.transform_nobbox = transform_nobbox

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_id = row['image_id']
        image_path = os.path.join(self.img_dir, image_id)
        print(image_path)
        
        image = Image.open(image_path).convert('RGB')
        # image = Image.open(r"C:\Users\Acer\Desktop\padchest_normalized.png").convert('RGB')
        image = np.array(image)

        label = row['label']
        disease_name = row['class_name']

        x_min, y_min, x_max, y_max = row[["x_min", "y_min", "x_max", "y_max"]]

        bbox = [x_min, y_min, x_max, y_max]
        
        if has_valid_bbox(bbox) and self.transform_bbox:
            augmented = self.transform_bbox(
                image=image,
                bboxes=[bbox],
                bbox_labels=[0]   # dummy label (required by Albumentations)
            )
        
            image = augmented["image"]
            bbox = augmented["bboxes"][0]
            # bbox = torch.tensor(bbox, dtype=torch.float32)
            has_bbox = torch.tensor(1, dtype=torch.bool)
            disease_id = disease2id[disease_name]
        
        else:
            augmented = self.transform_nobbox(image=image)
        
            image = augmented["image"]
            # bbox = torch.zeros(4, dtype=torch.float32)
            # bbox = [0, 0, 0, 0]
            has_bbox = torch.tensor(0, dtype=torch.bool)
            disease_id= disease2id['no_bbox']

        bbox = torch.tensor(bbox, dtype=torch.float32)
        
        # if any(pd.isna([x_min, y_min, x_max, y_max])):
            # bbox = torch.zeros(4, dtype=torch.float32)
            # has_bbox = torch.tensor(0, dtype=torch.bool)
            # disease_id= disease2id['no_bbox']
        # else:
            # bbox = torch.tensor([x_min, y_min, x_max, y_max], dtype=torch.float32)
            # has_bbox = torch.tensor(1, dtype=torch.bool)
            # disease_id = disease2id[disease_name]

        
        disease_id = torch.tensor(disease_id, dtype=torch.long)
        label = 0 if row['label'] == 'Normal' else 1
        label = torch.tensor(label, dtype=torch.float32)

        return image, disease_id, label, bbox, has_bbox

In [16]:
import torch
from multimodel import Multimodel

model = Multimodel().to(device)
model.load_state_dict(torch.load(checkpoint_path, map_location=device))

/home/jupyter-nafisha/.local/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jupyter-nafisha/.local/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<All keys matched successfully>

In [17]:
import torch
import torch.nn.functional as F

class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.activations = None
        self.gradients = None
        self._register_hooks()

    def _register_hooks(self):
        def forward_hook(module, input, output):
            self.activations = output
            output.register_hook(self._save_gradients)

        self.target_layer.register_forward_hook(forward_hook)

    def _save_gradients(self, grad):
        self.gradients = grad

    def generate(self, input_tensor):
        self.model.zero_grad()

        logits = self.model(input_tensor)      # shape: [1, 1]
        score = logits[:, 0]                   # binary logit

        score.backward(retain_graph=True)

        # Grad-CAM formula
        weights = self.gradients.mean(dim=(2, 3), keepdim=True)
        cam = (weights * self.activations).sum(dim=1, keepdim=True)

        cam = F.relu(cam)
        cam = F.interpolate(
            cam,
            size=input_tensor.shape[2:],
            mode="bilinear",
            align_corners=False
        )

        cam = cam[0, 0].detach().cpu().numpy()
        cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-8)

        return cam

In [18]:
import torch.nn as nn
def disable_inplace_relu(module):
    if isinstance(module, nn.ReLU):
        module.inplace = False

model.apply(disable_inplace_relu)
model.eval()

Multimodel(
  (backbone): DenseNet(
    (features): Sequential(
      (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU()
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU()
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU()
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLayer(
          (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_sta

In [44]:
from transform import val_transform, train_transform_bbox, train_transform_nobbox
# from dataset import XrayDataset
from torch.utils.data import DataLoader

img_dir = '/home/common/data_v3'
# test_csv = '/home/jupyter-nafisha/normal-abnormal-multitask/CSVs/test_withoutBbox.csv'
test_csv = '/home/jupyter-nafisha/normal-abnormal-multitask/CSVs/vinbig_test_withBBox.csv'
# test_csv = '/home/jupyter-nafisha/normal-abnormal-multitask/CSVs/train.csv'

# test_dataset = XrayDataset(img_dir, test_csv, transform_nobbox=val_transform)
test_dataset = XrayDataset(img_dir, test_csv, transform_bbox=train_transform_bbox, transform_nobbox=train_transform_nobbox)
dataset = DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=False,
    num_workers=4
)

In [45]:
def overlay_heatmap(cam, image, alpha=0.3):
    """
    cam: numpy array [H, W] in range [0,1]
    image: torch tensor [1, 3, H, W]
    """
    img = image.squeeze().permute(1, 2, 0).cpu().numpy()

    # normalize image to [0,255]
    img = (img - img.min()) / (img.max() - img.min())
    img = np.uint8(255 * img)

    cam = cv2.resize(cam, (img.shape[1], img.shape[0]))
    heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)

    overlay = cv2.addWeighted(img, 1 - alpha, heatmap, alpha, 0)
    return overlay

In [46]:
disease2id = {
    'no_bbox': 0,
    'No finding': 1,
    'Aortic enlargement': 2,
    'Atelectasis': 3,
    'Calcification': 4,
    'Cardiomegaly': 5,
    'Consolidation': 6,
    'ILD': 7,
    'Infiltration': 8,
    'Lung Opacity': 9,
    'Nodule/Mass': 10,
    'Other lesion': 11,
    'Pleural effusion': 12,
    'Pleural thickening': 13,
    'Pneumothorax': 14,
    'Pulmonary fibrosis': 15
}

In [47]:
def is_valid_bbox(bbox):
    if bbox is None:
        return False
    if torch.isnan(bbox).any():
        return False
    if torch.all(bbox == 0):
        return False
    return True

In [50]:
def draw_bbox(image, bbox, color=(255, 0, 0), thickness=1):
    """
    image: torch tensor [1, 3, H, W]
    bbox: tensor [4] -> x_min, y_min, x_max, y_max
    """
    img = image.squeeze().permute(1, 2, 0).cpu().numpy()

    img = (img - img.min()) / (img.max() - img.min())
    img = np.uint8(255 * img)

    x1, y1, x2, y2 = bbox.int().tolist()

    img = cv2.rectangle(
        img,
        (x1, y1),
        (x2, y2),
        color,
        thickness
    )
    return img

In [55]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

target_layer = model.backbone.features.norm5
gradcam = GradCAM(model, target_layer)

for i in range(10):
    image, disease_id, label, bbox, has_bbox = test_dataset[i]
    print(bbox)

    label = label.item()
    image = image.unsqueeze(0).to(device)

    # ---- Forward (NO torch.no_grad) ----
    logits = model(image)
    prob = torch.sigmoid(logits).item()
    pred = int(prob > 0.5)

    label_map = {0: "Normal", 1: "Abnormal"}
    print(f"Prediction: {label_map[pred]}, Actual: {label_map[label]}, Prob: {prob:.4f}")

    # ---- Grad-CAM ----
    cam = gradcam.generate(image)
    heatmap_overlay = overlay_heatmap(cam, image)
    print(heatmap_overlay.shape)

    # # ---- BBox ----
    bbox_img = None
    if is_valid_bbox(bbox):
        bbox_img = draw_bbox(image, bbox)

    # ---- Plot ----
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    if bbox_img is not None:
        plt.imshow(bbox_img)
        plt.title("Original + Bounding Box")
    else:
        plt.imshow(image.squeeze().permute(1, 2, 0).cpu())
        plt.title("Original (No BBox)")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(heatmap_overlay)
    plt.title("Original + Grad-CAM")
    plt.axis("off")

    plt.tight_layout()
    plt.show()
    break

/home/common/data_v3/vinbig/0c7a38f293d5f5e4846aa4ca6db4daf1.jpg
tensor([132.0473,  21.4795, 214.4910, 190.1589])
Prediction: Abnormal, Actual: Abnormal, Prob: 0.9455


/home/jupyter-nafisha/.local/lib/python3.12/site-packages/torch/_tensor.py:1002: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


RuntimeError: requested resize to (224, 224) ((224, 224) elements in total), but the given tensor has a size of 1x3x224x224 (150528 elements). autograd's resize can only change the shape of a given tensor, while preserving the number of elements. 

In [177]:
image, disease_id, label, bbox, has_bbox = test_dataset[0]  # choose IDX
print(disease_id, label)
image = image.unsqueeze(0).to(device)

tensor(0) tensor(1.)


In [56]:
import torch
import torch.nn.functional as F

class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.activations = None
        self.gradients = None
        self._register_hooks()

    def _register_hooks(self):
        def forward_hook(module, input, output):
            self.activations = output

        def backward_hook(module, grad_input, grad_output):
            self.gradients = grad_output[0]

        self.target_layer.register_forward_hook(forward_hook)
        self.target_layer.register_full_backward_hook(backward_hook)

    def generate(self, input_tensor):
        self.model.zero_grad()

        logits = self.model(input_tensor)      # [1, 1]
        score = logits[:, 0]                   # binary logit
        score.backward()

        # GAP over gradients
        weights = self.gradients.mean(dim=(2, 3), keepdim=True)

        cam = (weights * self.activations).sum(dim=1)
        cam = F.relu(cam)

        cam = F.interpolate(
            cam.unsqueeze(1),
            size=input_tensor.shape[2:],
            mode="bilinear",
            align_corners=False
        )

        cam = cam.squeeze().detach().cpu().numpy()

        # Proper normalization
        cam = cam - cam.min()
        cam = cam / (cam.max() + 1e-8)

        return cam


In [57]:
def overlay_heatmap(cam, image, alpha=0.4):
    """
    cam: [H, W] numpy (0–1)
    image: torch tensor [1, 3, H, W]
    """
    img = image.squeeze().permute(1, 2, 0).cpu().numpy()
    img = img - img.min()
    img = img / (img.max() + 1e-8)

    cam_uint8 = np.uint8(255 * cam)
    heatmap = cv2.applyColorMap(cam_uint8, cv2.COLORMAP_JET)
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
    heatmap = heatmap / 255.0

    overlay = np.clip((1 - alpha) * img + alpha * heatmap, 0, 1)
    return overlay


In [58]:
def is_valid_bbox(bbox):
    if bbox is None:
        return False
    if torch.isnan(bbox).any():
        return False
    if torch.all(bbox == 0):
        return False
    return True


def draw_bbox(image, bbox, color=(0, 255, 0), thickness=3):
    """
    image: torch tensor [1, 3, H, W]
    bbox: tensor [4] -> x_min, y_min, x_max, y_max
    """
    img = image.squeeze().permute(1, 2, 0).cpu().numpy()
    img = img - img.min()
    img = img / (img.max() + 1e-8)
    img = np.uint8(255 * img)

    x1, y1, x2, y2 = bbox.int().tolist()

    img = cv2.rectangle(
        img,
        (x1, y1),
        (x2, y2),
        color,
        thickness
    )
    return img


In [59]:
import matplotlib.pyplot as plt

target_layer = model.backbone.features.norm5
gradcam = GradCAM(model, target_layer)

model.eval()

for i in range(10):
    image, disease_id, label, bbox, has_bbox = test_dataset[i]

    label = label.item()
    image = image.unsqueeze(0).to(device)

    # ---- Forward (NO torch.no_grad) ----
    logits = model(image)
    prob = torch.sigmoid(logits).item()
    pred = int(prob > 0.5)

    label_map = {0: "Normal", 1: "Abnormal"}
    print(f"Prediction: {label_map[pred]}, Actual: {label_map[label]}, Prob: {prob:.4f}")

    # ---- Grad-CAM ----
    cam = gradcam.generate(image)
    heatmap_overlay = overlay_heatmap(cam, image)

    # ---- BBox ----
    bbox_img = None
    if is_valid_bbox(bbox):
        bbox_img = draw_bbox(image, bbox)

    # ---- Plot ----
    plt.figure(figsize=(12, 6))

    # Original + BBox
    plt.subplot(1, 2, 1)
    if bbox_img is not None:
        plt.imshow(bbox_img)
        plt.title("Original + Bounding Box")
    else:
        plt.imshow(image.squeeze().permute(1, 2, 0).cpu())
        plt.title("Original (No BBox)")
    plt.axis("off")

    # Original + Grad-CAM
    plt.subplot(1, 2, 2)
    plt.imshow(heatmap_overlay)
    plt.title("Original + Grad-CAM")
    plt.axis("off")

    plt.tight_layout()
    plt.show()

    break


/home/common/data_v3/vinbig/0c7a38f293d5f5e4846aa4ca6db4daf1.jpg


RuntimeError: Output 0 of BackwardHookFunctionBackward is a view and is being modified inplace. This view was created inside a custom Function (or because an input was returned as-is) and the autograd logic to handle view+inplace would override the custom backward associated with the custom Function, leading to incorrect gradients. This behavior is forbidden. You can fix this by cloning the output of the custom Function.